<a href="https://colab.research.google.com/github/FibGro/Pyspark/blob/main/Skewness_in_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=aee6f63aa9a5dfc0410b76cdd31c026b34e11c2eeba4752b47ea56b78101fb84
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("SkewinDF").getOrCreate()

In [3]:
# Loading Data Skew

import numpy as np
import pandas as pd
import random


# sale dataset:
# table 1: OrderID, Qty, Sales, Discount (yes=1, no=0)
# table 2: ProductID, OrderID, Product, Price

########### Table 1 ###########

key_1 = [101] * 100
key_2 = [201] * 7000000
key_3 = [301] * 500
key_4 = [401] * 10000
OrderID = key_1 + key_2 + key_3 + key_4
random.shuffle(OrderID)

Qty_1 = list(np.random.randint(low = 1, high = 100, size = len(key_1)))
Qty_2 = list(np.random.randint(low = 1, high = 200, size = len(key_2)))
Qty_3 = list(np.random.randint(low = 1, high = 1000, size = len(key_3)))
Qty_4 = list(np.random.randint(low = 1, high = 50, size = len(key_4)))
Qty = Qty_1 + Qty_2 + Qty_3 + Qty_4

Sales_1 = list(np.random.randint(low = 10, high = 100, size = len(key_1)))
Sales_2 = list(np.random.randint(low = 50, high = 3400, size = len(key_2)))
Sales_3 = list(np.random.randint(low = 12, high = 2000, size = len(key_3)))
Sales_4 = list(np.random.randint(low = 40, high = 1000, size = len(key_4)))
Sales = Sales_1 + Sales_2 + Sales_3 + Sales_4

Discount = list(np.random.randint(low = 0, high = 2, size = len(OrderID)))
data1 = list(zip(OrderID,Qty,Sales,Discount))

# Create the Pandas DF
data_skew = pd.DataFrame(data1, columns=['OrderID','Qty','Sales','Discount'])


########### Table 2 ###########
data2 = [[1, 101, 'pencil', 4.99],
         [2, 101, 'book', 9.5],
         [3, 101, 'scissors', 14],
         [4, 301, 'glue', 7],
         [5, 201, 'marker', 8.49],
         [6, 301, 'label', 2],
         [7, 201, 'calculator', 3.99],
         [8, 501, 'eraser', 1.55],
        ]

data_small = pd.DataFrame(data2, columns=['ProductID', 'OrderID', 'Product', 'Price'])

In [4]:
# Create PySpark DF from Pandas

# Optimize conversion between PySpark and Pandas DF: Enable arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

df_skew = spark.createDataFrame(data_skew)
df_skew.printSchema()
df_skew.show()
df_skew.rdd.getNumPartitions()

root
 |-- OrderID: long (nullable = true)
 |-- Qty: long (nullable = true)
 |-- Sales: long (nullable = true)
 |-- Discount: long (nullable = true)

+-------+---+-----+--------+
|OrderID|Qty|Sales|Discount|
+-------+---+-----+--------+
|    201| 70|   38|       1|
|    201| 87|   16|       1|
|    201| 86|   88|       0|
|    201| 25|   90|       0|
|    201| 47|   40|       1|
|    201| 63|   76|       0|
|    201| 46|   39|       0|
|    201| 42|   42|       1|
|    201| 95|   47|       0|
|    201| 58|   77|       1|
|    201| 65|   81|       1|
|    201| 74|   41|       1|
|    201|  5|   42|       1|
|    201| 95|   44|       0|
|    201| 14|   45|       1|
|    201| 17|   62|       1|
|    201| 34|   53|       0|
|    201| 77|   44|       1|
|    201| 16|   91|       0|
|    201| 77|   60|       1|
+-------+---+-----+--------+
only showing top 20 rows



702

In [5]:
df_small = spark.createDataFrame(data_small)
df_small.printSchema()
df_small.show()
df_small.rdd.getNumPartitions()

root
 |-- ProductID: long (nullable = true)
 |-- OrderID: long (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: double (nullable = true)

+---------+-------+----------+-----+
|ProductID|OrderID|   Product|Price|
+---------+-------+----------+-----+
|        1|    101|    pencil| 4.99|
|        2|    101|      book|  9.5|
|        3|    101|  scissors| 14.0|
|        4|    301|      glue|  7.0|
|        5|    201|    marker| 8.49|
|        6|    301|     label|  2.0|
|        7|    201|calculator| 3.99|
|        8|    501|    eraser| 1.55|
+---------+-------+----------+-----+



2

In [ ]:
# RUn shuffle JOin() with small sized data

In [9]:
joined_df = df_skew.join(df_small, df_skew.OrderID == df_small.OrderID, how = "inner")
# DF increases the parition number to 200 automatically when spark performs data shuffling (join, aggregation)



In [13]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

# run join()
joined_df = df_skew.join(df_small, df_skew.OrderID == df_small.OrderID, how = "inner")


In [14]:
# perform a join() and descriptive statistics on a skewed data
from pyspark.sql.functions import *


groups = df_skew.join(df_small, df_skew.OrderID == df_small.OrderID, how = "inner")

summary = groups.select(mean(groups.Sales).alias("AVG(Sales)"),
                        stddev(groups.Sales).alias("STD(Sales)"),
                        min(groups.Sales).alias("MIN(Sales)"),
                        max(groups.Sales).alias("MAX(Sales)"),
                       )
summary.show()

+------------------+-----------------+----------+----------+
|        AVG(Sales)|       STD(Sales)|MIN(Sales)|MAX(Sales)|
+------------------+-----------------+----------+----------+
|1722.8482428060252|967.6718604469153|        11|      3399|
+------------------+-----------------+----------+----------+



In [15]:
# Mitigate data skewness: SALTING

from pyspark.sql.functions import *

# add  random value [0 2]
df_skew_salting = df_skew.withColumn("_salt_", round(rand() * 2))
df_small_salting = df_small.withColumn("_salt_", round(rand() * 2))

df_skew_salting.show()
df_small_salting.show()

+-------+---+-----+--------+------+
|OrderID|Qty|Sales|Discount|_salt_|
+-------+---+-----+--------+------+
|    201| 70|   38|       1|   0.0|
|    201| 87|   16|       1|   2.0|
|    201| 86|   88|       0|   1.0|
|    201| 25|   90|       0|   0.0|
|    201| 47|   40|       1|   0.0|
|    201| 63|   76|       0|   2.0|
|    201| 46|   39|       0|   1.0|
|    201| 42|   42|       1|   1.0|
|    201| 95|   47|       0|   2.0|
|    201| 58|   77|       1|   1.0|
|    201| 65|   81|       1|   1.0|
|    201| 74|   41|       1|   2.0|
|    201|  5|   42|       1|   2.0|
|    201| 95|   44|       0|   1.0|
|    201| 14|   45|       1|   1.0|
|    201| 17|   62|       1|   1.0|
|    201| 34|   53|       0|   1.0|
|    201| 77|   44|       1|   1.0|
|    201| 16|   91|       0|   0.0|
|    201| 77|   60|       1|   1.0|
+-------+---+-----+--------+------+
only showing top 20 rows

+---------+-------+----------+-----+------+
|ProductID|OrderID|   Product|Price|_salt_|
+---------+-------+---

In [16]:
# repartition using _salt_:
df_skew_salting = df_skew_salting.repartition(3, "_salt_")
df_small_salting = df_small_salting.repartition(3, "_salt_")

In [18]:
# apply join()

df_skew_salting.drop("_salt_")
df_small_salting.drop("_salt_")


groups = df_skew_salting.join(df_small_salting, df_skew_salting.OrderID == df_small_salting.OrderID, how = "inner")

summary = groups.select(mean(groups.Sales).alias("AVG(Sales)"),
                        stddev(groups.Sales).alias("STD(Sales)"),
                        min(groups.Sales).alias("MIN(Sales)"),
                        max(groups.Sales).alias("MAX(Sales)"),
                       )
summary.show()

+------------------+-----------------+----------+----------+
|        AVG(Sales)|       STD(Sales)|MIN(Sales)|MAX(Sales)|
+------------------+-----------------+----------+----------+
|1722.8482428060252|967.6718604468763|        11|      3399|
+------------------+-----------------+----------+----------+

